In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay, accuracy_score

In [ ]:
train = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet").groupby('customer_ID').tail(4)
test = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet").groupby('customer_ID').tail(4)
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")

In [ ]:
# train = train_load.copy()
# test = test_load.copy()
# train_labels = train_labels_load.copy()

In [ ]:
# Drop columns that have more than 40 % missing values
columns = train.columns[(train.isna().sum() / len(train)) * 100 > 40]
train = train.drop(columns, axis=1)
test = test.drop(columns, axis=1)

In [ ]:
# Impute
train = train.bfill(axis='rows').ffill(axis='rows')
test = test.bfill(axis='rows').ffill(axis='rows')
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
train = train.groupby('customer_ID').tail(1)
test = test.groupby('customer_ID').tail(1)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
# Shape
train.shape, train_labels.shape, test.shape
obj_col = [
    'B_30', 'B_38', 
    'D_114', 'D_116', 
    'D_117', 'D_120', 
    'D_126', 'D_63', 
    'D_64', 'D_66', 
    'D_68']
train[obj_col].head

In [ ]:
obj_col = [
    'B_30', 'B_38', 
    'D_114', 'D_116', 
    'D_117', 'D_120', 
    'D_126', 'D_63', 
    'D_64', 'D_66', 
    'D_68']
for col in obj_col:
    train[col]=train[col].astype('int').astype('str')
    test[col]=test[col].astype('int').astype('str')
#     print(train[col].unique())
#     print(test[col].unique())
train[obj_col].head

In [ ]:
train = train.merge(train_labels, how='inner', on="customer_ID")

In [ ]:
test_data = test.copy()
train = train.drop(['index','customer_ID', 'S_2'], axis=1)
test = test.drop(['index','customer_ID', 'S_2'], axis=1)

In [ ]:
# Encode categorical features
train = pd.get_dummies(train, columns=obj_col, drop_first=True)
test = pd.get_dummies(test, columns=obj_col, drop_first=True)
train.shape, test.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(train)
train = scaling.transform(train)
scaling = MinMaxScaler(feature_range=(-1,1)).fit(test)
test = scaling.transform(test)

In [ ]:
Features = train.loc[:, test.columns]
target = train['target']

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(Features, target.ravel())

In [ ]:
test_data['prediction']=clf._predict_proba_lr(test_dataset)[:,1]

In [ ]:
test_data[['customer_ID','prediction']].to_csv("submission-svm.csv", index=False)

In [ ]:
test_data.head()